# REEpy - Data Visualization Notebook

This notebook demonstrates how to use the REEpy package to fetch and visualize electricity data from Red Eléctrica Española (REE).

## Setup

First, let's import the necessary libraries and initialize the REE client.

In [6]:
# Add the Reepy project directory to the Python path
import sys
import os

# Get the absolute path to the project root directory (parent of the notebooks directory)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the project root to the Python path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now you can import reepi modules
import reepi

In [7]:
# Make sure the REEpy package is installed
# If not installed, run: !pip install -e ..

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

# Import REEpy components
from reepi.api.client import REEClient
from reepi.utils.data_processing import (
    format_datetime,
    calculate_renewable_percentage,
    aggregate_by_type,
    calculate_daily_average,
    extract_time_series
)

# Initialize the client
client = REEClient()

/Users/ignasipascual/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 1. Fetching Electricity Generation Mix Data

Let's fetch data about the electricity generation mix for the past week.

In [8]:
# Define date range
end_date = datetime.now()
start_date = end_date - timedelta(days=7)

# Format dates for API
start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")

# Fetch generation mix data
generation_data = client.get_generation_mix(start_date_str, end_date_str, "hour")
generation_df = client.parse_generation_data(generation_data)
generation_df = format_datetime(generation_df)

# Display the first few rows
generation_df.head()

2025-05-05 18:56:10,742 - reepi.api.client - ERROR - Request error: 404 Client Error: Not Found for url: https://apidatos.ree.es/en/api/generation/evolution?start_date=2025-04-28&end_date=2025-05-05&time_trunc=hour


HTTPError: 404 Client Error: Not Found for url: https://apidatos.ree.es/en/api/generation/evolution?start_date=2025-04-28&end_date=2025-05-05&time_trunc=hour

### Visualizing Generation Mix

Now let's create some visualizations of the generation mix data.

In [ ]:
# Create a stacked area chart for generation mix
fig = px.area(
    generation_df, 
    x="datetime", 
    y="value", 
    color="type",
    title="Electricity Generation Mix (Past Week)",
    labels={"value": "Production (MW)", "datetime": "Date", "type": "Source"}
)

fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=600
)

fig.show()

### Calculating and Visualizing Renewable Percentage

In [ ]:
# Calculate renewable percentage over time
grouped_by_datetime = generation_df.groupby('datetime')
renewable_percentages = []

for dt, group in grouped_by_datetime:
    renewable_pct = calculate_renewable_percentage(group)
    renewable_percentages.append({'datetime': dt, 'renewable_percentage': renewable_pct})

renewable_df = pd.DataFrame(renewable_percentages)

# Plot renewable percentage over time
fig = px.line(
    renewable_df, 
    x="datetime", 
    y="renewable_percentage",
    title="Renewable Energy Percentage (Past Week)",
    labels={"renewable_percentage": "Percentage (%)", "datetime": "Date"}
)

fig.update_layout(height=500)
fig.show()

## 2. Fetching and Visualizing Electricity Prices

In [ ]:
# Fetch price data
price_data = client.get_electricity_prices(start_date_str, end_date_str, "hour")
price_df = client.parse_price_data(price_data)
price_df = format_datetime(price_df)

# Display the first few rows
price_df.head()

In [ ]:
# Plot electricity prices
fig = px.line(
    price_df, 
    x="datetime", 
    y="value", 
    color="type",
    title="Electricity Price Evolution (Past Week)",
    labels={"value": "Price (€/MWh)", "datetime": "Date", "type": "Market"}
)

fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=500
)

fig.show()

### Analyzing Price Statistics

In [ ]:
# Calculate price statistics by day
price_df['date'] = price_df['datetime'].dt.date
daily_stats = price_df.groupby(['date', 'type'])['value'].agg(['min', 'mean', 'max']).reset_index()

# Display stats
daily_stats.head(10)

In [ ]:
# Plot daily price ranges as a box plot
fig = px.box(
    price_df, 
    x="date", 
    y="value",
    color="type",
    title="Daily Price Range (Past Week)",
    labels={"value": "Price (€/MWh)", "date": "Date", "type": "Market"}
)

fig.update_layout(height=500)
fig.show()

## 3. Fetching and Visualizing Electricity Demand

In [ ]:
# Fetch demand data
demand_data = client.get_demand_data(start_date_str, end_date_str, "hour")
demand_df = extract_time_series(demand_data)
demand_df = format_datetime(demand_df)

# Display the first few rows
demand_df.head()

In [ ]:
# Plot demand data
fig = px.line(
    demand_df, 
    x="datetime", 
    y="value", 
    color="type",
    title="Electricity Demand (Past Week)",
    labels={"value": "Demand (MW)", "datetime": "Date", "type": "Type"}
)

fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=500
)

fig.show()

### Analyzing Demand Patterns

In [ ]:
# Add hour and weekday columns for pattern analysis
demand_df['hour'] = demand_df['datetime'].dt.hour
demand_df['weekday'] = demand_df['datetime'].dt.day_name()

# Create a heatmap of demand by hour and weekday
demand_type = demand_df['type'].unique()[0]  # Get one type for analysis
demand_pattern = demand_df[demand_df['type'] == demand_type].pivot_table(
    index='weekday', 
    columns='hour',
    values='value',
    aggfunc='mean'
)

# Sort weekdays
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
demand_pattern = demand_pattern.reindex(weekday_order)

# Create heatmap
fig = px.imshow(
    demand_pattern,
    labels=dict(x="Hour of Day", y="Day of Week", color="Demand (MW)"),
    x=demand_pattern.columns,
    y=demand_pattern.index,
    title=f"Average Demand Pattern by Hour and Day ({demand_type})"
)

fig.update_layout(height=500)
fig.show()

## 4. Advanced Analysis: Correlations Between Data Types

In [ ]:
# Prepare data for correlation analysis
# We'll need to aggregate data to a common time granularity

# Use hourly data
# Simplify to one type of price and demand for comparison
price_type = price_df['type'].unique()[0] if not price_df.empty else None
demand_type = demand_df['type'].unique()[0] if not demand_df.empty else None

# Extract simplified dataframes with just datetime and value
if price_type and demand_type:
    price_simple = price_df[price_df['type'] == price_type][['datetime', 'value']].rename(columns={'value': 'price'})
    demand_simple = demand_df[demand_df['type'] == demand_type][['datetime', 'value']].rename(columns={'value': 'demand'})
    
    # Merge dataframes on datetime
    corr_df = pd.merge(price_simple, demand_simple, on='datetime', how='inner')
    
    # Display merged data
    corr_df.head()
else:
    print("Missing price or demand data for correlation analysis")

In [ ]:
# Create scatter plot to visualize correlation
if 'corr_df' in locals() and not corr_df.empty:
    fig = px.scatter(
        corr_df, 
        x="demand", 
        y="price",
        trendline="ols",
        title=f"Correlation between Electricity Demand and Price",
        labels={"demand": f"Demand (MW) - {demand_type}", "price": f"Price (€/MWh) - {price_type}"}
    )
    
    fig.update_layout(height=500)
    fig.show()
    
    # Calculate correlation coefficient
    correlation = corr_df['demand'].corr(corr_df['price'])
    print(f"Correlation coefficient between demand and price: {correlation:.4f}")

## 5. CO2 Emissions Analysis

In [ ]:
# Fetch CO2 emissions data
emissions_data = client.get_co2_emissions(start_date_str, end_date_str, "day")
emissions_df = extract_time_series(emissions_data)
emissions_df = format_datetime(emissions_df)

# Display the first few rows
emissions_df.head()

In [ ]:
# Plot CO2 emissions
if not emissions_df.empty:
    fig = px.bar(
        emissions_df, 
        x="datetime", 
        y="value", 
        color="type",
        title="CO2 Emissions from Electricity Generation",
        labels={"value": "CO2 (tCO2eq)", "datetime": "Date", "type": "Type"}
    )
    
    fig.update_layout(
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        height=500
    )
    
    fig.show()
else:
    print("No emissions data available for the selected period.")

## 6. Exporting Data for Further Analysis

You can export the data to CSV files for further analysis in other tools.

In [ ]:
# Export data to CSV files
# Define export directory
import os
export_dir = "../data"
os.makedirs(export_dir, exist_ok=True)

# Export dataframes
if not generation_df.empty:
    generation_df.to_csv(f"{export_dir}/generation_mix_{start_date_str}_to_{end_date_str}.csv", index=False)
    print(f"Generation data exported to {export_dir}/generation_mix_{start_date_str}_to_{end_date_str}.csv")
    
if not price_df.empty:
    price_df.to_csv(f"{export_dir}/electricity_prices_{start_date_str}_to_{end_date_str}.csv", index=False)
    print(f"Price data exported to {export_dir}/electricity_prices_{start_date_str}_to_{end_date_str}.csv")
    
if not demand_df.empty:
    demand_df.to_csv(f"{export_dir}/electricity_demand_{start_date_str}_to_{end_date_str}.csv", index=False)
    print(f"Demand data exported to {export_dir}/electricity_demand_{start_date_str}_to_{end_date_str}.csv")
    
if not emissions_df.empty:
    emissions_df.to_csv(f"{export_dir}/co2_emissions_{start_date_str}_to_{end_date_str}.csv", index=False)
    print(f"Emissions data exported to {export_dir}/co2_emissions_{start_date_str}_to_{end_date_str}.csv")

## 7. Conclusion

This notebook demonstrates how to use the REEpy package to fetch and visualize electricity data from Red Eléctrica Española (REE). You can extend this analysis by:

1. Analyzing longer time periods to identify seasonal patterns
2. Building forecasting models for electricity demand or prices
3. Comparing renewable energy growth over time
4. Analyzing the relationship between weather data and generation/demand

For more information about the REE API, visit: [https://www.ree.es/en/apidatos](https://www.ree.es/en/apidatos)